In [1]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents.strategies import TerminationStrategy

In [2]:
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.functions import kernel_function

from typing import Annotated

In [3]:
import json
import os
from dotenv import load_dotenv
import psycopg2
from psycopg2.extras import RealDictCursor
load_dotenv()

True

In [4]:
def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    kernel = Kernel()
    kernel.add_service(AzureChatCompletion(service_id=service_id, 
                                        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
                                        deployment_name=os.getenv("AZURE_OPENAI_CHAT_COMPLETION_MODEL"),
                                        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                                        ))
    return kernel

In [5]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "approved" in history[-1].content.lower()

In [6]:
from decimal import Decimal

class DatabaseConnector:
    def __init__(self):
        self.create_connection()

    @kernel_function(description="Create a connection object to the postgres database.")
    def create_connection(self):
        print("create_connection function called... ")
        connection = psycopg2.connect(
            dbname=os.getenv("DB_NAME"),
            user=os.getenv("DB_USER"),
            password=os.getenv("DB_PASSWORD"),
            host=os.getenv("DB_HOST"),
            port=os.getenv("DB_PORT")
        )
        self.connection = connection
        self.cursor = self.connection.cursor(cursor_factory=RealDictCursor)

    @kernel_function(description="Fetches user information for a given account id.")
    def query_database(self, query: Annotated[str, "query to be executed"]) -> Annotated[str, "Returns the queried information as a json"]:
        """
        Fetches the information from the required table in PostgreSQL database.

        :param query (str): the query to be executed.
        :return: fetched information as a JSON string.
        :rtype: str
        """
        print("query_database function called... query: ", query)
        try:
            # if(self.connection != True):
            #     self.create_connection()
            
            #connection = self.connection
            cursor = self.cursor
            cursor.execute(query=query)
            result_record = cursor.fetchone()
            if result_record:
                # Convert Decimal values to strings
                for key, value in result_record.items():
                    if isinstance(value, Decimal):
                        result_record[key] = str(value)
                return json.dumps({"result_record": result_record})
            else:
                return json.dumps({"error": "An error occured while fetching the data."})
        except Exception as e:
            return json.dumps({"error": str(e)})
        # finally:
        #     if connection:
        #         self.close_connection()
        
    
    @kernel_function(description="Closes the connection to the database.")
    def close_connection(self) -> Annotated[str, "Returns a message indicating the status of the connection closure."]:
        """
        Closes the connection to the PostgreSQL database.

        :return: Message indicating the status of the connection closure.
        :rtype: str
        """
        print("close_connection function called... ")
        try:
            if self.connection:
                self.cursor.close()
                self.connection.close()
            return "Connection closed successfully."
        except Exception as e:
            return str(e)

In [7]:
acc = DatabaseConnector()
acc.create_connection()
print(acc.query_database("select * from customerdata where customer_id = '136743'"))
acc.close_connection()

create_connection function called... 
create_connection function called... 
query_database function called... query:  select * from customerdata where customer_id = '136743'
{"result_record": {"customer_id": 136743, "card_blocked": true, "payment_due": false, "card_type": "MasterCard", "credit_card_no": 1234567812345678}}
close_connection function called... 


'Connection closed successfully.'

In [8]:
def _create_kernel_with_chat_completion_and_plugin(service_id: str) -> Kernel:
    kernel = Kernel()
    kernel.add_service(AzureChatCompletion(service_id=service_id, 
                                        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
                                        deployment_name=os.getenv("AZURE_OPENAI_CHAT_COMPLETION_MODEL"),
                                        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                                        ))
    kernel.add_plugin(DatabaseConnector(), plugin_name="db_connector")
    return kernel

In [9]:
TRANSCRIPTION_REVIEWER = "TranscriptionReviewer"
TRANSCRIPTION_REVIEWER_INSTRUCTIONS = """
As a transcription reviewer, your task is to review a conversation between a customer service agent and a customer. 
Your goal is to determine if the customer's card is blocked.

Follow these steps:

Examine the conversation transcript to check if the customer's card is blocked.
If the card is blocked, extract the customer ID from the transcript and provide it to the Orchestrator.
If the card is not blocked, provide a response stating that you are skipping this transcript as it is not related to Card Blocking.
"""

ANALYST_NAME = "BusinessAnalyst"
ANALYST_INSTRUCTIONS = """
As a highly skilled business analyst with extensive experience in writing and executing SQL queries, your task is to determine the reason for a blocked card by executing necessary SQL queries. 

Follow these best practices:
1. Fix SQL Errors: Ensure all SQL queries are syntactically correct and free of errors.
2. Execute Queries: Do not make any assumptions about query results. Execute the necessary SQL queries to obtain accurate information.

You have access to a PostgreSQL database containing information about customers and their credit card transactions. The database includes two tables:
- `customerdata`: Columns include `customer_id`: integer, `card_blocked`: boolean, `payment_due`: boolean, `card_type`: varchar, and `credit_card_no`: bigint.
- `credit_card_transactions`: Columns include `credit_card_no`: varchar, `date`: timestamp, `amount`: decimal, `authentication_passed`: boolean, and `location`: varchar.

You will receive a customer ID from the Orchestrator. Follow these steps to determine the reason for the blocked card:
1. Check the `card_blocked` column in the `customerdata` table to see if the card is blocked.
2. If the card is not blocked, inform the Orchestrator that the card is not blocked and the transcript is being skipped as it is not related to Card Blocking.
3. If the card is blocked, check the `credit_card_transactions` table for more than three authentication failures recently.
4. If there are more than three authentication failures recently, the card is blocked due to authentication failures. 
5. If there are zero or less than 3 authentication failures, the card is blocked due to an unknown reason. 

Provide the reason (authentication failure or unknown) to the Orchestrator.

"""

ORCHESTRATOR_NAME = "Orchestrator"
ORCHESTRATOR_INSTRUCTIONS = """
As an orchestrator, your task is to coordinate the work of the transcription reviewer and the business analyst to determine the reason for a blocked card and communicate the findings to the customer.

Follow these steps:
1. Receive the customer ID from the transcription reviewer.
2. Pass the customer ID to the business analyst and request them to determine the reason for the blocked card by executing the necessary SQL queries.
3. If the transcription reviewer states that the card is not blocked, conclude that the transcript is being skipped as it is not related to card blocking.
4. After receiving the analysis with a known reason for the issue from the business analyst, draft an email to the customer with the analysis provided.
5. If the reason is unknown, email the customer success team (copying the customer) that this needs to be further analyzed by them.
5. Approve the analysis by using keywords such as "approved" or "not approved" after drafting the email.
6. If the analysis is not approved, provide feedback to the business analyst for further improvements.

Best Practices:
- Ensure clear and concise communication between the transcription reviewer and the business analyst.
- Verify the syntax of the SQL queries executed by the business analyst and point out any errors.
- When asking a question to the business analyst or transcription reviewer, be specific and clear.
- Maintain a professional and empathetic tone in the email to the customer.
- Double-check the analysis before approving it to ensure it is thorough and accurate.

"""

TASK = "Read transcript and determine the reason for blocked card"

In [10]:
# 1. Create the reviewer agent based on the chat completion service
agent_reviewer = ChatCompletionAgent(
    kernel=_create_kernel_with_chat_completion("TranscriptionReviewer"),
    name=TRANSCRIPTION_REVIEWER,
    instructions=TRANSCRIPTION_REVIEWER_INSTRUCTIONS,
)

In [11]:
# 2. Create the copywriter agent based on the chat completion service
agent_analyst = ChatCompletionAgent(
    kernel=_create_kernel_with_chat_completion_and_plugin("BusinessAnalyst"),
    name=ANALYST_NAME,
    instructions=ANALYST_INSTRUCTIONS,
    
)

create_connection function called... 


In [12]:
# 3. Create the reviewer agent based on the chat completion service
agent_orchestrator = ChatCompletionAgent(
    kernel=_create_kernel_with_chat_completion("Orchestrator"),
    name=ORCHESTRATOR_NAME,
    instructions=ORCHESTRATOR_INSTRUCTIONS,
)

In [13]:
# 4. Place the agents in a group chat with a custom termination strategy
group_chat = AgentGroupChat(
    agents=[
        agent_reviewer,
        agent_analyst,
        agent_orchestrator 
        
    ],
    termination_strategy=ApprovalTerminationStrategy(
        agents=[agent_orchestrator],
        maximum_iterations=100,
    ),
)

In [14]:
from semantic_kernel.agents.group_chat.agent_chat import ChatMessageContent

message_content = ChatMessageContent(role="user", content=""" 
                                     Customer ID : 136743
Date: 16-03-2025 10:29:30

Call Transcript:            
            
Agent: how can I help you today? 
Customer: My card is blocked, could you please help me with that?
Agent: I am sorry to hear that. Can you please provide me with your name, contact number, email id and address to unblock the card?
Customer: My name is Vikas, contact number is 1234567890, email id is vsdsdf@gmail.com and address is 1234, 5th Avenue, New York, NY 10001
Agent: Thank you for providing the details. I have raised a request for this to be looked at immediately. You will receive a confirmation email shortly. Is there anything else I can help you with?
Customer: No, that is all. Thank you.
Agent: You're welcome. Have a great day!                                     
                                     """)

In [15]:
# 4. Add the task as a message to the group chat
await group_chat.add_chat_message(message_content)


In [16]:
# 5. Invoke the chat
async for content in group_chat.invoke():
    print(f"# {content.name}: {content.content}")


# TranscriptionReviewer: The customer's card is blocked. The customer ID is **136743**.

Providing this information to the Orchestrator.
create_connection function called... 
query_database function called... query:  SELECT card_blocked FROM customerdata WHERE customer_id = 136743;
query_database function called... query:  SELECT COUNT(*) AS auth_failures_recent FROM credit_card_transactions WHERE credit_card_no = (SELECT credit_card_no FROM customerdata WHERE customer_id = 136743) AND authentication_passed = false AND date >= NOW() - interval '30 days';
query_database function called... query:  SELECT COUNT(*) AS auth_failures_recent FROM credit_card_transactions WHERE credit_card_no = CAST((SELECT credit_card_no FROM customerdata WHERE customer_id = 136743) AS varchar) AND authentication_passed = false AND date >= NOW() - interval '30 days';
close_connection function called... 
# BusinessAnalyst: The database transaction was aborted due to an error that occurred during the query exec